In [1]:
import scipy
import numpy as np
from sympy.utilities.iterables import multiset_partitions, partitions
from itertools import permutations
# plot the bn values
import matplotlib.pyplot as plt
from tqdm import tqdm
import jax.numpy as jnp
import jax
import math
from tqdm import tqdm

# Computing the norm of $\|\Psi^{[2s]}_{m,p}\|$

In [2]:
factorial = {}
for i in range(100):
    factorial[i] = np.longdouble(math.factorial(i))

We define the parameters we are interested in:

In [11]:
s = 3
maxp = 20
m = 6
h = 1/(4*m) #1/(4*m)
if m == 2:
    assert(s==2)
    cs = [[1/2, 1/6], [1/2, 1/6]]
if m == 3:
    assert(s==2)
    cs = [[0, 1/12], [1, 0], [0, 1/12]]
elif m == 5:
    assert(s==3)
    cs = [[0.2, 0.08734395950888931101, 0.03734395950888931101], 
        [0.34815492558797391479, 0.053438272547684150, 0.00584269157837031012],
        [abs(1-2*(0.2+0.34815492558797391479)), 0, abs(1/12-2*(0.03734395950888931101 + 0.00584269157837031012))],
        [0.34815492558797391479, 0.053438272547684150, 0.00584269157837031012],
        [0.2, 0.08734395950888931101, 0.03734395950888931101]]
elif m == 6:
    assert(s==3)
    cs = [[0.208, 0.09023186422416794596, 0.03823186422416794596], 
        [0.312, 0.04467385661651479788, 0.00439421553992544024],
        [abs(1/2-(0.208 + 0.09023186422416794596)), 0.01407960659498524468, abs(1/24-(0.03823186422416794596+0.00439421553992544024))],
        [abs(1/2-(0.208 + 0.09023186422416794596)), 0.01407960659498524468, abs(1/24-(0.03823186422416794596+0.00439421553992544024))],
        [0.312, 0.04467385661651479788, 0.00439421553992544024],
        [0.208, 0.09023186422416794596, 0.03823186422416794596]]
elif m == 11:
    assert(s==4)
    cs = [
        [0.169715531043933180094151, 0.152866146944615909929839, 0.119167378745981369601216, 0.068619226448029559107538],
        [0.379420807516005431504230, 0.148839980923180990943008, 0.115880829186628075021088, 0.188555246668412628269760],
        [0.469459306644050573017994, 0.379844237839363505173921, 0.022898814729462898505141, 0.571855043580130805495594],
        [0.448225927391070886302766, 0.362889857410989942809900, 0.022565582830528472333301, 0.544507517141613383517695],
        [0.293924473106317605373923, 0.026255628265819381983204, 0.096761509131620390100068, 0.000018330145571671744069],
        [0.447109510586798614120629, 0, 0.200762581179816221704073, 0],
        [0.293924473106317605373923, 0.026255628265819381983204, 0.096761509131620390100068, 0.000018330145571671744069],
        [0.448225927391070886302766, 0.362889857410989942809900, 0.022565582830528472333301, 0.544507517141613383517695],
        [0.469459306644050573017994, 0.379844237839363505173921, 0.022898814729462898505141, 0.571855043580130805495594],
        [0.379420807516005431504230, 0.148839980923180990943008, 0.115880829186628075021088, 0.188555246668412628269760],
        [0.169715531043933180094151, 0.152866146944615909929839, 0.119167378745981369601216, 0.068619226448029559107538]
        ]

We need two key functions `efficient_number_compositions` and `efficient_weak_compositions`

In [4]:
def efficient_number_compositions(p, s):
    r"""
    p: integers up to which compositions add up
    s: maximum value of each term in the composition

    Computes a dictionary where the key is the length of the composition,
    and the value is the number of compositions of that length.
    """
    pts = partitions(p, k=s)
    
    comps = {}
    for part in pts:
        len_ = np.sum(list(part.values()))

        num_comps = factorial[len_]
        for n in part.values():
            num_comps /= factorial[n]

        if len_ in comps.keys():
            comps[len_] += num_comps
        else:
            comps[len_] = num_comps

    return comps

In [32]:
def partition_to_list(part):
    l = []
    for k, v in part.items():
        l += [k]*v
    return l


def add_term(m, mean_cs, part):
    part_ = partition_to_list(part)
    part_ = part_ + [0]*(m-len(part_))
    perms = permutations(part_)
                
    add_term = np.longdouble(0)
    for per in perms:
        add_term += np.prod([mean_cs[i]**per[i]/factorial[per[i]] for i in range(len(per))])
    return add_term

def efficient_weak_compositions(maxw, list_m, list_cs, use_max = True):
    r"""
    Computes sum{k_1...k_m\in weak_compositions} c^{k_1}/k_1!...c^{k_m}/k_m!) for all weak compositions of length m.
    Depending on the value of `use_max`, it uses either the maximum value of c for all at once (faster) or the specific values of c,
    (more accurate).

    maxw: maximum value of the sum of the weak composition
    list_m: list of the possible values of m.
    list_cs: list of the possible values of cs.
    use_max: if True, use the maximum value of cs instead of the specific values of cs. Faster but less accurate.
    """

    weak_comps = {}
    for m, cs in zip(list_m, list_cs):

        vector_add_term = np.vectorize(lambda part: add_term(m, mean_cs, part))

        max_cs = np.max(cs)
        mean_cs = np.mean(cs, axis = 1)

        weak_comps[m] = {}
        for normw in range(maxw+1):
            
            suma = np.longdouble(0)

            pts = partitions(normw, m=m)

            if use_max:
                for part in pts:
                    len_ = np.sum(list(part.values()))

                    num_comps = factorial[m]/factorial[m-len_] * max_cs**len_
                    for n in part.values():
                        num_comps /= factorial[n]

                    denominator = np.longdouble(1)
                    for i in part.keys():
                        denominator = denominator*factorial[i]

                    suma = suma + num_comps/denominator

            else:
                add = vector_add_term(list(pts))
                suma += np.sum(add)

            weak_comps[m][normw] = suma

    return weak_comps


maxw = 20
weak_comps = efficient_weak_compositions(maxw, list_m = [m], list_cs = [cs], use_max=False)
for normw in range(maxw+1):
    print(f'm={m}, w={normw}, result={weak_comps[m][normw]}')

m=6, w=0, result=720.0
m=6, w=1, result=73.14711648584196
m=6, w=2, result=11.242167771290237
m=6, w=3, result=1.2640311468138534
m=6, w=4, result=0.14928528672831104
m=6, w=5, result=0.015443591264099124
m=6, w=6, result=0.0016469290671441046
m=6, w=7, result=9.720275231769182e-05
m=6, w=8, result=6.704968142211123e-06
m=6, w=9, result=3.7910828615795877e-07
m=6, w=10, result=2.2072913098487436e-08
m=6, w=11, result=1.0731278791184764e-09
m=6, w=12, result=5.5680185646304536e-11
m=6, w=13, result=2.177074819955771e-12
m=6, w=14, result=9.173694447291485e-14
m=6, w=15, result=3.4025395370427856e-15
m=6, w=16, result=1.2602620426565485e-16
m=6, w=17, result=4.138755674323608e-18
m=6, w=18, result=1.4172907380280127e-19
m=6, w=19, result=4.074224450288789e-21
m=6, w=20, result=1.2234001899344244e-22


Final calculation

In [7]:
def summatory_compositions(maxp, s, list_m, list_cs, use_max = True): #todo: better choose between maximum and actual value
    r"""
    maxp: maximum order p of the summatory
    s: 2s is the order of the Commutator Free Magnus operator
    maxc: maximum value of the norm of |x_{i,j}a_j|
    cs: list of values |x_{i,j}a_j| for each i,j

    Note: you should choose one of cs and maxc, not both.
    """

    weak_comps = efficient_weak_compositions(maxp, list_m, list_cs, use_max)

    suma = {}
    for p in range(1, maxp+1):
        num_comps = efficient_number_compositions(p, s)

        suma[p] = np.longdouble(0)
        for len_comp, nc in num_comps.items():
            suma[p] += nc*weak_comps[m][len_comp] * h**p

    return suma

sum_compositions = summatory_compositions(maxp, s, [m], [cs], use_max = True)

#accumulate sum_compositions
acc_sum_compositions = {}
acc_sum_compositions[0] = 0
for i in range(1, maxp+1):
    acc_sum_compositions[i] = acc_sum_compositions[i-1] + sum_compositions[i]


for i in range(1, maxp+1):
    print(f'{i}: {sum_compositions[i]}, {acc_sum_compositions[i]}')

1: 0.08703873139699347, 0.08703873139699347
2: 0.009558201160053703, 0.09659693255704718
3: 0.0009787548700926263, 0.0975756874271398
4: 8.630859781959907e-05, 0.0976619960249594
5: 7.556410239295036e-06, 0.09766955243519869
6: 6.413007132720286e-07, 0.09767019373591196
7: 5.21406942001763e-08, 0.09767024587660617
8: 4.141176091661513e-09, 0.09767025001778226
9: 3.2045407312889726e-10, 0.09767025033823634
10: 2.4318208473478705e-11, 0.09767025036255456
11: 1.8155396782699394e-12, 0.09767025036437009
12: 1.3369537784265144e-13, 0.09767025036450379
13: 9.740230435300974e-15, 0.09767025036451353
14: 7.038638596730466e-16, 0.09767025036451424
15: 5.0464542234809233e-17, 0.0976702503645143
16: 3.587815632517645e-18, 0.0976702503645143
17: 2.528712955093948e-19, 0.0976702503645143
18: 1.7680571474251283e-20, 0.0976702503645143
19: 1.2277620102284523e-21, 0.0976702503645143
20: 8.47432929480125e-23, 0.0976702503645143


Accumulating the error over all orders $p>2s+1$

In [8]:
def acc_from(i, sum_compositions):
    
    acc_sum_compositions = {}
    acc_sum_compositions[i-1] = 0
    for i in range(i, maxp+1):
        acc_sum_compositions[i] = acc_sum_compositions[i-1] + sum_compositions[i]

    return acc_sum_compositions

acc_sum_compositions = acc_from(2*s+1, sum_compositions)
for i in range(2*s+1, maxp+1):
    print(f'{i}: {acc_sum_compositions[i]}')

7: 5.21406942001763e-08
8: 5.628187029183781e-08
9: 5.660232436496671e-08
10: 5.662664257344019e-08
11: 5.6628458113118454e-08
12: 5.66285918084963e-08
13: 5.6628601548726736e-08
14: 5.6628602252590594e-08
15: 5.6628602303055136e-08
16: 5.662860230664295e-08
17: 5.662860230689582e-08
18: 5.6628602306913504e-08
19: 5.6628602306914735e-08
20: 5.662860230691482e-08


We can compare using both modes in, for $s = 3$ and $m = 5$ and $m = 6:

In [12]:
sum_compositions = summatory_compositions(maxp, s, [m], [cs], use_max = True)

acc_sum_compositions = acc_from(2*s+1, sum_compositions)
for i in range(2*s+1, maxp+1):
    print(f'{i}: {acc_sum_compositions[i]}')

7: 1.9826761441780353e-08
8: 2.1195662281119414e-08
9: 2.1287833070435784e-08
10: 2.129391745212139e-08
11: 2.1294312404704615e-08
12: 2.1294337682074482e-08
13: 2.1294339281894766e-08
14: 2.1294339382273752e-08
15: 2.129433938852205e-08
16: 2.1294339388907903e-08
17: 2.129433938893154e-08
18: 2.1294339388932976e-08
19: 2.1294339388933062e-08
20: 2.129433938893307e-08


In [31]:
sum_compositions = summatory_compositions(maxp, s, [m], [cs], use_max = False)

acc_sum_compositions = acc_from(2*s+1, sum_compositions)
for i in range(2*s+1, maxp+1):
    print(f'{i}: {acc_sum_compositions[i]}')

7: 2.2270680814019967e-09
8: 2.2918155165031245e-09
9: 2.293493412952103e-09
10: 2.293531840383346e-09
11: 2.293532836672026e-09
12: 2.293532860563087e-09
13: 2.293532861107771e-09
14: 2.2935328611205704e-09
15: 2.2935328611208475e-09
16: 2.2935328611208533e-09
17: 2.2935328611208533e-09
18: 2.2935328611208533e-09
19: 2.2935328611208533e-09
20: 2.2935328611208533e-09
